In [34]:
import numpy as np
import pandas as pd
import itertools
import csv
import nltk

In [35]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [36]:
###Extracting sentences from csv file
with open("reddit.csv", "r", encoding="utf-8") as file:
    read = csv.reader(file, skipinitialspace = True)
    next(read)
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in read])
    sentences = ["%s %s %s"%(sentence_start_token,x,sentence_end_token) for x in sentences]
    print("number of parsed sentences {}".format(len(sentences)))
        

number of parsed sentences 79170


In [37]:
###Vocabulary info
wordtokens = nltk.FreqDist(itertools.chain(*[nltk.word_tokenize(x) for x in sentences]))

In [38]:
###most common words
vocab = wordtokens.most_common(vocabulary_size-1)

In [25]:
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [17]:
print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

Using vocabulary size 8000.
The least frequent word in our vocabulary is 'documentary' and appeared 10 times.


In [18]:
###replacing words not in vocab as unknown_token
tokenised_sent = [nltk.word_tokenize(x) for x in sentences]

In [20]:
for i,sent in enumerate(tokenised_sent):
    tokenised_sent[i] = [w if w in index_to_word else unknown_token for w in sent ]

In [26]:
print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenised_sent[0])


Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'


In [27]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenised_sent])
Y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenised_sent])

"SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END"

[0,
 6,
 3494,
 7,
 155,
 795,
 25,
 222,
 8,
 32,
 20,
 202,
 4954,
 350,
 91,
 6,
 66,
 207,
 5,
 2]